<img width="50px" align="left" style="margin-right:20px" src="http://data.newprolab.com/public-newprolab-com/npl_logo.png"> <b>New Professions Lab</b> <br /> Специалист по большим данным

# Решение проекта 1

# Спрогнозировать пол и возрастную категорию интернет-пользователей по логу посещения сайтов

<img width="110px" align="left" src="http://data.newprolab.com/public-newprolab-com/project01_img0.png?img">

Одна из задач DMP-системы состоит в том, чтобы по разрозненным даннным, таким, как посещения неким пользователем сайтов, классифицировать его и присвоить ему определённую категорию: пол, возраст, интересы и так далее. В дальнейшем составляется портрет, или профиль, пользователя, на основе которого ему более таргетированно показывается реклама в интернете.

### Задача

Используя доступный набор данных о посещении страниц у одной части пользователей, сделать прогноз относительно **пола и возрастной категории** другой части пользователей. Угадывание (hit) - правильное предсказание и пола, и возрастной категории одновременно.


In [85]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os, sys
import json
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote

### Обработка данных на вход

Загрузим весь датасет в pandas:

In [86]:
data_dir = '/data/share/project01'
filename = 'gender_age_dataset.txt'
file_path = '/'.join([data_dir,filename])
df = pd.read_csv(file_path, sep='\t')

И теперь попробуем понять, что у нас есть:

In [59]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


Скомбинируем пол и возраст вединую категорию

In [87]:
df['target'] = df[['gender','age']].apply(lambda x: x['gender']+x['age'], axis=1)
df['target'].unique()

array(['F18-24', 'M25-34', 'F25-34', 'M>=55', 'F45-54', 'F35-44',
       'M35-44', 'F>=55', 'M18-24', 'M45-54', '--'], dtype=object)

Что содержится в `user_json`?

In [88]:
df.iloc[0].user_json

'{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}'

Видим, что это некая сериализованная json-строка, которую можно легко разобрать через модуль `json`:

In [89]:
j = json.loads(df.iloc[0].user_json)
j

{'visits': [{'url': 'http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun',
   'timestamp': 1419688144068},
  {'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story',
   'timestamp': 1426666298001},
  {'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html',
   'timestamp': 1426666298000},
  {'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story',
   'timestamp': 1426661722001},
  {'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html',
   'timestamp': 1426661722000}]}

### Очистка данных и feature engineering

Одна из первых вещей, которые можно попробовать — это вытащить домены и использовать их в качестве признаков. Можно воспользоваться функцией:

In [90]:
def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    #if netloc is not None: return str(netloc.encode('utf8')).strip()
    if netloc is not None: return str(netloc.strip())
    return None

In [91]:
def get_domains(j):
    return " ".join([url2domain(d['url']) for d in json.loads(j)['visits'] ])

In [92]:
df['user_json'].head().apply(get_domains, 0)

0    zebra-zoya.ru news.yandex.ru sotovik.ru news.y...
1    sweetrading.ru sweetrading.ru sweetrading.ru 1...
2    ru.oriflame.com ru.oriflame.com ru.oriflame.co...
3    translate-tattoo.ru nadietah.ru 1obl.ru 1obl.r...
4    mail.rambler.ru news.rambler.ru mail.rambler.r...
Name: user_json, dtype: object

In [93]:
%%time
df['urls'] = df['user_json'].apply(get_domains, 0)

CPU times: user 1min 11s, sys: 224 ms, total: 1min 11s
Wall time: 1min 11s


### Деление на train и test сеты, обучение модели, предсказания для test-сета

Давайте теперь оценим размер нашего train и test сетов. Train set:

In [94]:
len(df[~((df.gender == '-') & (df.age == '-'))])

36138

Test set:

In [95]:
len(df[(df.gender == '-') & (df.age == '-')])

5000

In [69]:
len(df) # Весь датасет

41138

In [96]:
df_train = df[~((df.gender == '-') & (df.age == '-'))]
df_test = df[(df.gender == '-') & (df.age == '-')]

In [97]:
df_train.head()

,gender,age,uid,user_json,target,urls
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000...",F18-24,zebra-zoya.ru news.yandex.ru sotovik.ru news.y...
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=...",M25-34,sweetrading.ru sweetrading.ru sweetrading.ru 1...
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr...",F25-34,ru.oriflame.com ru.oriflame.com ru.oriflame.co...
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r...",F25-34,translate-tattoo.ru nadietah.ru 1obl.ru 1obl.r...
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#...",M>=55,mail.rambler.ru news.rambler.ru mail.rambler.r...


In [98]:
df_test.head()

,gender,age,uid,user_json,target,urls
36138,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,"{""visits"": [{""url"": ""http://www.interfax.ru/bu...",--,interfax.ru amerikan-gruzovik.ru amerikan-gruz...
36139,-,-,bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd,"{""visits"": [{""url"": ""https://www.packagetrackr...",--,packagetrackr.com packagetrackr.com packagetra...
36140,-,-,bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c,"{""visits"": [{""url"": ""http://www.mk.ru/incident...",--,mk.ru mk.ru mk.ru m.news.yandex.ru
36141,-,-,bd7c5d7a-0def-41d1-895f-fdb96c56c2d4,"{""visits"": [{""url"": ""http://www.24open.ru/user...",--,24open.ru ohotniki.ru ohotniki.ru google.ru oh...
36142,-,-,bd7e54a2-0215-45cb-a869-9efebf250e38,"{""visits"": [{""url"": ""http://www.dns-shop.ru/ca...",--,dns-shop.ru easyhelp.ru primpogoda.ru primpogo...


### TFIdf

Воспользуемся известной нам моделью. По умолчанию, встроенный токенайзер разобьет нам домены по точке, поэтому даем модели опцию, которая описывает регулярным выражением, как выделять токены. В нашем случае, токены - все что не пробелы.
Вторая опция позволяет не считать число посещений каждого сайта, а просто выставлять флаг 1, если сайт был посещен. Она сработала лучше, чем дефолтная False.

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    token_pattern = '\S+',
    binary=True,
)

Мы пока только объявили модель, но не обучили. 

### Naive Bayes

Наивный баейсовский классификатор хорошо зарекомендовал себя в спам-фильтрах. Мы используем многоклассовую его версию. В качестве добавочного бенефита - он работает гораздо быстрее логистической регрессии.

In [100]:
from sklearn.naive_bayes import MultinomialNB

In [101]:
nb = MultinomialNB()

## Pipeline

Pipeline позволет нам построить сложную модель, состоящую из нескольких шагов-преобразований данных и использовать ее как простую. Разберите примеры из документации этого класса 
http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
а так же кернел на Kaggle для каких-то продвинутых трюков
https://www.kaggle.com/metadist/work-like-a-pro-with-pipelines-and-feature-unions

Определяем пайплайн

In [102]:
from sklearn.pipeline import Pipeline

model = Pipeline([
    ('tfdif', tfidf),
    ('nb', nb),
])

Обучаем эту составную модель

In [103]:
model.fit(df_train['urls'],df_train['target'])

Pipeline(memory=None,
     steps=[('tfdif', TfidfVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...True,
        vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [104]:
pred = model.predict(df_test['urls'])

In [105]:
df_test['gender'] = [s[:1] for s in pred]

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [106]:
df_test['age'] = [s[1:] for s in pred]

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [81]:
df_test['age'].head()

36138    25-34
36139    25-34
36140    25-34
36141    25-34
36142    25-34
Name: age, dtype: object

In [107]:
df_test

,gender,age,uid,user_json,target,urls
36138,M,25-34,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,"{""visits"": [{""url"": ""http://www.interfax.ru/bu...",--,interfax.ru amerikan-gruzovik.ru amerikan-gruz...
36139,M,25-34,bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd,"{""visits"": [{""url"": ""https://www.packagetrackr...",--,packagetrackr.com packagetrackr.com packagetra...
36140,M,25-34,bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c,"{""visits"": [{""url"": ""http://www.mk.ru/incident...",--,mk.ru mk.ru mk.ru m.news.yandex.ru
36141,M,25-34,bd7c5d7a-0def-41d1-895f-fdb96c56c2d4,"{""visits"": [{""url"": ""http://www.24open.ru/user...",--,24open.ru ohotniki.ru ohotniki.ru google.ru oh...
36142,M,25-34,bd7e54a2-0215-45cb-a869-9efebf250e38,"{""visits"": [{""url"": ""http://www.dns-shop.ru/ca...",--,dns-shop.ru easyhelp.ru primpogoda.ru primpogo...
36143,M,35-44,bd7e9797-4cdb-46e1-a540-f3ea010605ad,"{""visits"": [{""url"": ""http://news.meta.ua/clust...",--,news.meta.ua
36144,M,25-34,bd7e9ec7-fb67-45eb-8ad3-209d01d15ae6,"{""visits"": [{""url"": ""http://dynamobryansk.foru...",--,dynamobryansk.forum24.ru
36145,F,25-34,bd8056df-cc25-4b63-bc12-a46f888baa49,"{""visits"": [{""url"": ""http://www.2mm.ru/mzdorov...",--,2mm.ru medlinks.ru
36146,F,25-34,bd818690-73d2-445d-be5d-5c8f748dbb19,"{""visits"": [{""url"": ""http://www.lacywear.ru/go...",--,lacywear.ru lacywear.ru cache.betweendigital.com
36147,M,25-34,bd81e006-f059-4cdd-b716-3467c78d1312,"{""visits"": [{""url"": ""http://nn.domru.ru/"", ""ti...",--,nn.domru.ru


In [108]:
outf = "out1.json"
#outf = sys.stdout
df_test[['uid', 'gender', 'age']]\
    .sort_values(by='uid',axis = 0, ascending = True)\
    .to_json(outf, orient='records')

### Сохранение модели

In [84]:
import pickle
    
with open('/data/home/oksana.strashynskaya/project01_model.pickle', 'wb') as f:
    pickle.dump(model, f)


Для самопроверки вы можете локально оттестировать свой скрипт, используя следующую команду:

In [21]:
#!tail -n1000 gender_age_dataset.txt | python3 project01_gender-age.py > output.json

### Сhecker

Приводится тут для наглядности. Можете сделать copy-paste в `project01_gender-age.py` и заменить строчки для ввода-вывода на те, где используется stdin/stdout

In [22]:
import pandas as pd
import re, json
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote

def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']: return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    #if netloc is not None: return str(netloc.encode('utf8')).strip()
    if netloc is not None: return str(netloc.strip())
    return None

def get_domains(j):
    return " ".join([url2domain(d['url']) for d in json.loads(j)['visits'] ])

#считываем данные
data = pd.DataFrame(columns=['gender','age','uid','user_json'])
s=0
import sys
#for line in sys.stdin:
test_lines = !tail -n5000 $file_path
for line in test_lines:
    data.loc[s] = line.strip().split("\t")
    s+=1

In [23]:
# преобразование данных
data['urls'] = data['user_json'].apply(get_domains, 0)

In [24]:
# считывание модели
import pickle

with open('/data/home/oksana.strashynskaya/project01_model.pickle', 'rb') as f:
    model = pickle.load( f)    

# предсказание
pred = model.predict(data['urls'])
data['gender'] = [s[:1] for s in pred]
data['age'] = [s[1:] for s in pred]

# вывод
outf = "out1.json"
#outf = sys.stdout
data[['uid', 'gender', 'age']]\
    .sort_values(by='uid',axis = 0, ascending = True)\
    .to_json(outf, orient='records')

Это решение дает 0.303 на паблике.